# ShopAssist AI

#### Project Background

## Introduction

In today's digital age, online shopping has become the go-to option for many consumers. However, the overwhelming number of choices and the lack of personalized assistance can make the shopping experience daunting. To address this, we have developed **ShopAssist AI, a chatbot that combines the power of large language models and rule-based functions to ensure accurate and reliable information delivery**.


#### Problem Statement

*Given a dataset containing information about laptops (product names, specifications, descriptions, etc.), build a chatbot that parses the dataset and provides accurate laptop recommendations based on user requirements*.


You can load the data and see it here.

#### Approach:

1. **Conversation and Information Gathering**: The chatbot will utilize language models to understand and generate natural responses. Through a conversational flow, it will ask relevant questions to gather information about the user's requirements.
2. **Information Extraction**: Once the essential information is collected, rule-based functions come into play, extracting top 3 laptops that best matches the user's needs.
3. **Personalized Recommendation**: Leveraging this extracted information, the chatbot engages in further dialogue with the user, efficiently addressing their queries and aiding them in finding the perfect laptop solution.

## System Design

#### Dataset:
We have a dataset `laptop.csv` where  each row describes the features of a single laptop and also has a small description at the end. The chatbot that we build will leverage LLMs to parse this `Description` column and provide recommendations

The chatbot should ask a series of questions to
- Determine the user's requirments. For simplicity, we have used 6 features to encapsulate the user's needs. The 6 features are as follows:
    - GPU intensity
    - Display quality
    - Portability
    - Multitasking
    - Processing speed
    - Budget

- Confirm if the user's requirements have been correctly captured at the end.

After that the chatbot lists down the top 3 products that are the most relevant, and engages in further conversation to help the user find the best one.

#### Building the Chatbot

Now let's go ahead and understand the system design for the chatbot.

![Chatbot_sys_design.png](https://drive.google.com/uc?id=1j-mw_dNcbxGcelQ0PmkDB0nKOpauU1wX)

`Stage 1`
- Intent Clarity Layer
- Intent Confirmation Layer
- Restart the conversation if change in Intent

`Stage 2`

- Product Mapping Layer
- Product Information Extraction Layer

`Stage 3`

- Product Recommendation Layer

##### Major functions behind the Chatbot

Let's now look at a brief overview of the major functions that form the chatbot. We'll take a deep dive later



- `initialize_conversation()`: This initializes the variable conversation with the system message.
- `get_chat_completions()`: This takes the ongoing conversation as the input and returns the response by the assistant
- `moderation_check()`: This checks if the user's or the assistant's message is inappropriate. If any of these is inappropriate, it ends the conversation.
- `intent_confirmation_layer()`: This function takes the assistant's response and evaluates if the chatbot has captured the user's profile clearly. Specifically, this checks if the following properties for the user has been captured or not GPU intensity, Display quality, Portability, Multitasking, Processing speed, Budget
- `is_intent_changed()`: Detects if the user changes the primary purpose or key preferences mid-conversation (e.g., from gaming to programming).Compares new user input to previous confirmed intents. If there's a shift in purpose or a major change in specs, it resets the conversation context and restarts the recommendation flow.
- `compare_laptops_with_user()`: This function compares the user's profile with the different laptops and come back with the top 3 recommendations.
- `initialize_conv_reco()`: Initializes the recommendations conversation

## Implementation:

### Import the libraries

Let's start by importing the libraries that we'll require for this project. Following are the ones:
- openai
- pandas
- os, json, ast

Make sure the api key is stored in the text file `OPENAI_API_Key.txt`.

In [ ]:
# Install OpenAI library
!pip install -U -q openai tenacity

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 723.4/723.4 kB 10.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import the libraries
import os, json, ast
import pandas as pd
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt
from google.colab import userdata

In [ ]:
# Set the display width to control the output width
pd.set_option('display.width', 100)
# Read the dataset and read the Laptop Dataset
df = pd.read_csv('laptop_data.csv')
df.head()

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...


In [ ]:
# If you're using the default OpenAI API key, uncomment the following lines:
os.environ['OPENAI_API_KEY'] = userdata.get('OPEN_API_KEY')

In [ ]:
## Function Description for the Function Calling API's

function_descriptions = [
    {
        "name": "compare_laptops_with_user",
        "description": "Get the top 3 laptops from the catalogue that best match the user's preferences based on GPU intensity, display quality, portability, multitasking, processing speed, and budget.",
        "parameters": {
            "type": "object",
            "properties": {
                "gpu_intensity": {
                    "type": "string",
                    "enum": ["low", "medium", "high"],
                    "description": "GPU capacity requirement classified as low, medium, or high."
                },
                "display_quality": {
                    "type": "string",
                    "enum": ["low", "medium", "high"],
                    "description": "Display quality requirement classified as low, medium, or high."
                },
                "portability": {
                    "type": "string",
                    "enum": ["low", "medium", "high"],
                    "description": "Portability requirement classified as low, medium, or high."
                },
                "multitasking": {
                    "type": "string",
                    "enum": ["low", "medium", "high"],
                    "description": "Multitasking requirement classified as low, medium, or high."
                },
                "processing_speed": {
                    "type": "string",
                    "enum": ["low", "medium", "high"],
                    "description": "Processing speed requirement classified as low, medium, or high."
                },
                "budget": {
                    "type": "integer",
                    "description": "The user's maximum budget."
                }
            },
            "required": [
                "gpu_intensity", "display_quality", "portability",
                "multitasking", "processing_speed", "budget"
            ]
        }
    },
    {
        "name": "intent_confirmation_layer",
        "description": "Evaluates if the user input contains all six required fields with valid values. Returns 'Yes' or 'No' in 'result', and an optional reason if 'No'.",
        "parameters": {
            "type": "object",
            "properties": {
                "response_assistant": {
                    "type": "string",
                    "description": "The assistant's interpretation of the user's hardware requirements."
                }
            },
            "required": ["response_assistant"]
        }
    },
    {
       "name": "product_map_layer",
        "description": "Extracts key laptop specs from description and classifies them into categories like GPU intensity, display quality, etc.",
        "parameters": {
        "type": "object",
        "properties": {
            "laptop_description": {
                "type": "string",
                "description": "The detailed textual description of the laptop, including specifications like CPU, GPU, RAM, display, and weight."
            }
        },
        "required": ["laptop_description"]
    }
    },
    {
    "name": "initialize_conv_reco",
    "description": "Starts a conversation summarizing laptop products returned as recommendations. Accepts zero, one, or more laptops and returns a conversation formatted for chat completion.",
    "parameters": {
        "type": "object",
        "properties": {
            "products": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "name": {"type": "string", "description": "Laptop name"},
                        "specs": {"type": "string", "description": "Major specifications summary"},
                        "price": {"type": "integer", "description": "Price in Indian Rupees"}
                    },
                    "required": ["name", "specs", "price"]
                },
                "description": "List of recommended laptops with name, specs, and price"
            }
        },
        "required": ["products"]
    }
    }
]


### `initialize_conversation()`:
This initializes the variable conversation with the system message. Using prompt engineering and chain of thought reasoning, the function will enable thechatbot to keep asking questions until the user requirements have been captured in a dictionary. It also includes Few Shot Prompting(sample conversation between the user and assistant) to align the model about user and assistant responses at each step.

In [ ]:
def initialize_conversation():
    delimiter = "####"

    system_message = f"""
    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.
    You are trying to understand the user's requirement for these laptop's features: ('gpu intensity','display quality','portability','multitasking','processing speed','budget')
    You need to ask relevant questions and understand the user need for each feature by analysing the user's responses.
    After understanding their requirements, you'll use a function call to suggest the top 3 laptops with their respective user match score.
    Recommend these laptops and answer any user's query about them.

    {delimiter} Here are certain guidelines that you need to follow:
    Don't ask questions about more than 2 features at a time.
    If the user's budget is less than says less than 25000 INR, please mention that there are no laptops in that range.
    Recommend the top3 laptops in the following format:
    Start with a brief summary of each laptop in the following format, in decreasing order of price of laptops:
    1. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    2. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    3. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    {delimiter}

    {delimiter}To find the top3 laptops, you need to have the following chain of thoughts:
    Thought 1: Ask one question to understand the user's profile and requirements. \n
    If their primary use for the laptop is unclear. Ask another question to comprehend their needs.
    Answer "Yes" or "No" to indicate if you understand the requirements. \n
    If yes, proceed to the next step. Otherwise, rephrase the question to capture their profile. \n

    Thought 2: Now, you are trying to understand the requirements for other features which you couldn't in the previous step.
    Ask questions to strengthen your understanding of the user's profile.
    Don't ask questions about more than 2 features at a time.
    Answer "Yes" or "No" to indicate if you understood all the needs of the features and are confident about the same.
    If yes, move to the next Thought. If no, ask question on the features whose needs you are unsure of. \n
    It is a good practice to ask question with a sound logic as opposed to directly citing the feature you want to understand the need for.{delimiter}
    {delimiter}

    {delimiter} Here is a sample conversation between the user and assistant:
    Assistant: "Hello! I'm here to help you find the perfect laptop that suits your needs. Could you please share your requirements?"
    User: "Hi, I am an editor."
    Assistant: "Great! As an editor, you likely require a laptop that can handle demanding tasks. Hence, the laptop should have high multitasking capability. You would also need a high end display for better visuals and editing. May I know what kind of work do you primarily focus on? Are you more involved in video editing, photo editing, or both? Understanding the specific type of editing work will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now."
    User: "I primarily work with After Effects."
    Assistant: "Thank you for providing that information. Working with After Effects involves working with graphics, animations, and rendering, which will require high GPU. Do you work with high-resolution media files, such as 4K videos or RAW photos? Understanding your file sizes will help determine the storage capacity and processing power needed."
    User: "Yes, sometimes I work with 4K videos as well."
    Assistant: "Thank you for the information. Processing 4K vidoes will require a good processor and high GPU. I think we have already determined earlier that you need a high GPU. To ensure I have a complete understanding of your needs, I have one more question: Are you frequently on the go and require a laptop that is lightweight and easy to carry, or do you primarily work from a stationary location?"
    User: "Yes, sometimes I travel but do not carry my laptop."
    Assistant:"Could you kindly let me know your budget for the laptop? This will help me find options that fit within your price range while meeting the specified requirements."
    User: "my max budget is 1.5lakh inr"
    {delimiter}


    Start with a short welcome message and encourage the user to share their requirements.
    """
    conversation = [{"role": "system", "content": system_message }]
    return conversation

### `get_chat_completions()`:
This takes the ongoing conversation as the input and returns the response by the assistant. We'll use the Chat Completions function for performing LLM calls to OpenAI.

In [ ]:
def get_chat_model_completions(messages):
    try:
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
            temperature=0,
            max_tokens=500,
            seed=2345,
            #tools=function_descriptions,
            #tool_choice="auto",
        )
        return response.choices[0].message  # return the full message object
    except openai.OpenAIError as e:
        print(f"OpenAI API error: {e}")
        return None

In [ ]:
# Testing the OpenAI functions defined above
input_prompt ='What is the capital of Italy?'
messages = [{'role':'user','content':input_prompt}]
# system_message_json_output = """<<. Return output in JSON format.>>"""
# messages[0]['content']+=system_message_json_output
messages

[{'role': 'user', 'content': 'What is the capital of Italy?'}]

In [ ]:
## Get LLM Outputs - normal
get_chat_model_completions(messages) ## Chat Completions API

ChatCompletionMessage(content='The capital of Italy is Rome.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)

### `moderation_check()`:
 This checks if the user's or the assistant's message is inappropriate. If any of these is inappropriate, you can add a break statement to end the conversation.

In [ ]:
# Define a function called moderation_check that takes user_input as a parameter.

def moderation_check(text_input):
    # Call the OpenAI API to perform moderation on the user's input.
    response = openai.moderations.create(input=text_input)
    # print(response)
    # Extract the moderation result from the API response.
    moderation_output = response.results[0].flagged
    # Check if the input was flagged by the moderation system.
    if moderation_output == True:
        # If flagged, return "Flagged"
        return "Flagged"
    else:
        # If not flagged, return "Not Flagged"
        return "Not Flagged"

### `intent_confirmation_layer()`:

This function takes the assistant's response and evaluates if the chatbot has captured the user's profile clearly. Specifically, this checks if the following properties for the user has been captured or not
   - GPU intensity
   - Display quality
   - Portability
   - Multitasking
   - Processing speed
   - Budget

In [ ]:
def intent_confirmation_layer(response_assistant):
    allowed_values = {'low', 'medium', 'high'}
    required_keys = [
        'GPU intensity', 'Display quality', 'Portability',
        'Multitasking', 'Processing speed', 'Budget'
    ]
    try:
        user_input = json.loads(response_assistant)
        for key in required_keys:
            if key not in user_input:
                return {"result": "No", "reason": f"Missing key: {key}"}
            value = user_input[key]
            if key == 'Budget':
                if not isinstance(value, int):
                    return {"result": "No", "reason": "Budget must be a number."}
            else:
                if value not in allowed_values:
                    return {"result": "No", "reason": f"Invalid value for {key}: {value}"}
        return {"result": "Yes"}
    except Exception as e:
        return {"result": "No", "reason": str(e)}

### Implementing the Product Mapping and Information Extraction Layers

In this section, we take in the output of the previous layers, i.e. the user requirements, which is in the format of a Python dictionary.
<br>
Next we will extract the top 3 laptop recommendations based on user's requirements.

This stage consists of the following helper functions that will implement the information extraction and product matching layers.

### `product_map_layer()`:

This function is responsible for extracting key features and criteria from laptop descriptions. Here's a breakdown of how it works:

-  Use a prompt that assign it the role of a Laptop Specifications Classifier, whose objective is to extract key features and classify them based on laptop descriptions.

- Provide step-by-step instructions for extracting laptop features from description.

- Assign specific rules for each feature (e.g., GPU Intensity, Display Quality, Portability, Multitasking, Processing Speed) and associate them with the appropriate classification value (Low, Medium, or High).

- Includes Few Shot Prompting (sample conversation between the user and assistant) to demonstrate the expected result of the feature extraction and classification process.

In [ ]:
def product_map_layer(laptop_description):
    delimiter = "#####"

    lap_spec = {
        "GPU intensity": "(Type of the Graphics Processor)",
        "Display quality": "(Display Type, Screen Resolution, Display Size)",
        "Portability": "(Laptop Weight)",
        "Multitasking": "(RAM Size)",
        "Processing speed": "(CPU Type, Core, Clock Speed)"
    }

    values = {'low', 'medium', 'high'}

    prompt = f"""
    You are a Laptop Specifications Classifier whose job is to extract the key features of laptops and classify them as per their requirements.
    To analyze each laptop, perform the following steps:
    Step 1: Extract the laptop's primary features from the description {laptop_description}
    Step 2: Store the extracted features in {lap_spec}
    Step 3: Classify each of the items in {lap_spec} into {values} based on the following rules:
    {delimiter}
    GPU Intensity:
    - low: <<< if GPU is entry-level such as an integrated graphics processor or entry-level dedicated graphics like Intel UHD >>> ,
    - medium: <<< if mid-range dedicated graphics like M1, AMD Radeon, Intel Iris >>> ,
    - high: <<< high-end dedicated graphics like Nvidia RTX >>> ,

    Display Quality:
    - low: <<< if resolution is below Full HD (e.g., 1366x768). >>> ,
    - medium: <<< if Full HD resolution (1920x1080) or higher. >>> ,
    - high: <<< if High-resolution display (e.g., 4K, Retina) with excellent color accuracy and features like HDR support. >>>

    Portability:
    - high: <<< if laptop weight is less than 1.51 kg >>> ,
    - medium: <<< if laptop weight is between 1.51 kg and 2.51 kg >>> ,
    - low: <<< if laptop weight is greater than 2.51 kg >>>

    Multitasking:
    - low: <<< If RAM size is 8 GB, 12 GB >>> ,
    - medium: <<< if RAM size is 16 GB >>> ,
    - high: <<< if RAM size is 32 GB, 64 GB >>>

    Processing Speed:
    - low: <<< if entry-level processors like Intel Core i3, AMD Ryzen 3 >>> ,
    - medium: <<< if Mid-range processors like Intel Core i5, AMD Ryzen 5 >>> ,
    - high: <<< if High-performance processors like Intel Core i7, AMD Ryzen 7 or higher >>>
    {delimiter}

    {delimiter}
    Here is input output pair for few-shot learning:
    input 1: "The Dell Inspiron is a versatile laptop that combines powerful performance and affordability. It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing. With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity. The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience. Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage. Additionally, it boasts an Intel UHD GPU for decent graphical performance and a backlit keyboard for enhanced typing convenience. With a one-year warranty and a battery life of up to 6 hours, the Dell Inspiron is a reliable companion for work or entertainment. All these features are packed at an affordable price of 35,000, making it an excellent choice for budget-conscious users."
    output 1: {{'GPU intensity': 'medium','Display quality':'medium','Portability':'medium','Multitasking':'high','Processing speed':'medium'}}

    {delimiter}
    ### Strictly don't keep any other text in the values of the JSON dictionary other than low or medium or high ###
    """

    input_msg = f"""Follow the above instructions step-by-step and output the dictionary in JSON format {lap_spec} for the following laptop {laptop_description}."""

    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": input_msg}
    ]

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",  # Supports response_format
        messages=messages,
        response_format = { "type": "json_object"}
    )

    return json.loads(response.choices[0].message.content)

In [ ]:
laptop_description_1 = f"""
The Dell Inspiron is a versatile laptop that combines powerful performance and affordability.
It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing.
With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity.
The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience.
Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage.
Additionally, it boasts an Intel UHD GPU for decent graphical performance and a backlit keyboard for enhanced typing convenience.
With a one-year warranty and a battery life of up to 6 hours, the Dell Inspiron is a reliable companion for work or entertainment.
All these features are packed at an affordable price of 35,000, making it an excellent choice for budget-conscious users.
"""

In [ ]:
display(product_map_layer(laptop_description_1))

{'GPU intensity': 'medium',
 'Display quality': 'medium',
 'Portability': 'medium',
 'Multitasking': 'high',
 'Processing speed': 'medium'}

Let's now apply this function to the entire laptop dataset

In [ ]:
##Run this code once to extract product info in the form of a dictionary
laptop_df= pd.read_csv('laptop_data.csv')

## Create a new column "laptop_feature" that contains the dictionary of the product features
laptop_df['laptop_feature'] = laptop_df['Description'].apply(lambda x: product_map_layer(x))

In [ ]:
laptop_df.to_csv("updated_laptop.csv",index=False,header = True)

### `compare_laptops_with_user()`:

This function compares the user's profile with the different laptops and come back with the top  recommendations. It will perform the following steps:
    - It will take the user requirements dictionary as input
    - Filter the laptops based on their price, keeping only the ones within the user's budget.
    - Calculate a score for each laptop based on how well it matches the user's requirements.
    - Sort the laptops based on their scores in descending order.
    - Return the top 3 laptops as a JSON-formatted string.

In [ ]:
import re

def parse_budget(budget_str):
    # Remove commas and non-digit characters
    clean_str = re.sub(r"[^\d]", "", str(budget_str))
    return int(clean_str) if clean_str else 0

In [ ]:
def extract_dictionary_from_string(string):
    regex_pattern = r"\{[^{}]+\}"

    dictionary_matches = re.findall(regex_pattern, string)

    # Extract the first dictionary match and convert it to lowercase
    if dictionary_matches:
        dictionary_string = dictionary_matches[0]
        dictionary_string = dictionary_string.lower()

        # Convert the dictionary string to a dictionary object using ast.literal_eval()
        dictionary = ast.literal_eval(dictionary_string)
    return dictionary

def compare_laptops_with_user(user_req_string):
  budget = parse_budget(user_req_string.get("budget", 0))
  #budget = user_req_string.get('budget', '0')
  laptop_df= pd.read_csv('updated_laptop.csv')
  filtered_laptops = laptop_df.copy()
  filtered_laptops['Price'] = filtered_laptops['Price'].str.replace(',','').astype(int)
  filtered_laptops = filtered_laptops[filtered_laptops['Price'] <= budget].copy()

  mappings = {
          'low': 0,
          'medium': 1,
          'high': 2
      }
  # Create 'Score' column in the DataFrame and initialize to 0
  filtered_laptops['Score'] = 0
  for index, row in filtered_laptops.iterrows():
      user_product_match_str = row['laptop_feature']
      laptop_values = extract_dictionary_from_string(user_product_match_str)
      score = 0

      for key, user_value in user_req_string.items():
        if key.lower() == 'budget':
            continue  # Skip budget comparison
        laptop_value = laptop_values.get(key, None)
        laptop_mapping = mappings.get(laptop_value.lower(), -1)
        user_mapping = mappings.get(user_value.lower(), -1)
        if laptop_mapping >= user_mapping:
          ### If the laptop value is greater than or equal to the user value the score is incremented by 1
          score += 1

      filtered_laptops.loc[index, 'Score'] = score

  # Sort the laptops by score in descending order and return the top 5 products
  top_laptops = filtered_laptops.drop('laptop_feature', axis=1)
  top_laptops = top_laptops.sort_values('Score', ascending=False).head(3)

  return top_laptops.to_json(orient='records')


def recommendation_validation(laptop_recommendation):

    data = json.loads(laptop_recommendation)
    data1 = []
    for i in range(len(data)):
      if data[i]['Score'] > 2:
        data1.append(data[i])
    return json.dumps(data1)

### Product Recommendation Layer

Finally, we come to the product recommendation layer. It takes the output from the `compare_laptops_with_user` function in the previous layer and provides the recommendations to the user. It has the following steps.
1. ialize the conversation for recommendation.
2. Generate the recommendations and display in a presentable format.
3. Ask questions basis the recommendations.

In [ ]:
def initialize_conv_reco(products):
    system_message = f"""
    You are an intelligent laptop gadget expert and you are tasked with the objective to \
    solve the user queries about any product from the catalogue in the user message \
    You should keep the user profile in mind while answering the questions.\

    Start with a brief summary of each laptop in the following format, in decreasing order of price of laptops:
    1. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    2. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>

    """
    user_message = f""" These are the user's products: {products}"""
    conversation = [{"role": "system", "content": system_message },
                    {"role":"user","content":user_message}]
    # conversation_final = conversation[0]['content']
    return conversation

### `is_intent_changed()`:

- Detects if the user changes the primary purpose or key preferences mid-conversation (e.g., from gaming to programming).
- How: Compares new user input to previous confirmed intents. If there's a shift in purpose or a major change in specs, it resets the conversation context and restarts the recommendation flow.

In [ ]:
def is_intent_changed(old_intent, new_intent):
    """
    Compare two user intents and return True if purpose or specs have changed significantly.
    """
    if not old_intent or not new_intent:
        return False

    keys_to_check = ['purpose', 'GPU intensity', 'Display quality', 'Portability', 'Multitasking', 'Processing speed']
    changes = sum(1 for key in keys_to_check if old_intent.get(key) != new_intent.get(key))

    return changes >= 2  # Allow tolerance for 1 minor change

In [ ]:
def dialogue_mgmt_system():
    # Step 1: Initialize conversation
    conversation = initialize_conversation()
    introduction = get_chat_model_completions(conversation).content
    print(introduction + '\n')

    user_input = ""
    current_intent = None

    while user_input.lower() != "exit":
        user_input = input()

        # Step 2: Moderation check for user input
        moderation = moderation_check(user_input)
        if moderation == 'Flagged':
            display("Sorry, this message has been flagged. Please restart your conversation.")
            break

        if user_input.lower() == "exit":
            print("Assistant: Goodbye! Feel free to return anytime for laptop advice.")
            break

        # Step 3: Add user input to conversation
        conversation.append({"role": "user", "content": user_input})

        # Step 4: Get assistant response
        response_assistant = get_chat_model_completions(conversation)

        # Step 5: Moderation check for assistant output
        moderation = moderation_check(response_assistant.content) # Pass only the content
        if moderation == 'Flagged':
            display("Sorry, this message has been flagged. Please restart your conversation.")
            break

        # Step 6: Handle function calls
        if response_assistant.function_call:
            print("\nAssistant: Got it! Let me analyze your needs...\n")

            # Step 7: Extract intent
            function_args = json.loads(response_assistant.function_call.arguments)
            new_intent = intent_confirmation_layer(json.dumps(function_args))

            # Step 8: Compare with current intent
            if current_intent and is_intent_changed(current_intent, new_intent):
                print("Assistant: Looks like your laptop needs have changed. Let's start fresh!\n")
                conversation = initialize_conversation()
                conversation.append({"role": "user", "content": user_input})
                current_intent = None
                continue

            current_intent = new_intent
            print(f"Assistant: Here's how I understood your needs:\n{new_intent}\n")

            # Step 9: Recommend laptops
            top_3_laptops = compare_laptops_with_user(current_intent)
            function_response = recommendation_validation(top_3_laptops)
            print(f"your 3 products:\n{function_response}\n")
            conversation_reco = initialize_conv_reco(function_response)

            # Step 10: Handle empty recommendations
            if len(conversation_reco) == 0:
                print("Assistant: Sorry, no laptops matched your requirements. Let me connect you with a human expert.")
                break

            # Step 11: Send function response to GPT
            conversation.append(response_assistant)
            conversation.append({
                "role": "function",
                "name": response_assistant.function_call.name,
                "content": conversation_reco,
            })

            # Step 12: Final assistant response
            final_response = get_chat_model_completions(conversation)
            conversation.append({"role": "assistant", "content": final_response.content})
            print("\nAssistant:\n" + final_response.content + "\n")

        else:
            # Step 13: Normal assistant message
            conversation.append({"role": "assistant", "content": response_assistant.content})
            print("\nAssistant:\n" + response_assistant.content + "\n")

In [ ]:
dialogue_mgmt_system()


    Hello! I'm here to help you find the perfect laptop that suits your needs. Could you please share your requirements?

Hi

Assistant:
Hello! I'm here to help you find the perfect laptop that suits your needs. Could you please share your requirements?

I am looking to purchase laptop for Gaming

Assistant:
Great! As a gamer, you likely require a laptop with high GPU intensity for smooth gaming experience. You would also need a high-quality display for better visuals and a good processing speed for seamless gameplay. May I know if you prioritize portability for gaming on the go, or do you primarily use the laptop in a stationary location? Understanding your gaming habits will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now.

I prefer to carry always

Assistant:
Got it! Portability is a key factor for you. Now, let's talk about the budget. What is your budget for the laptop? This will help me find options that fit within your price r